# Introduction


This notebook demonstrates how BioThings Explorer can be used to execute queries having more than one intermediate nodes:

The query starts from drug "Anisindione", the two intermediate nodes with be *Gene and DiseaseOrPhenotypicFeature", the final output will be "PhenotypicFeature".


**Background**: BioThings Explorer can answer two classes of queries -- "EXPLAIN" and "PREDICT".  EXPLAIN queries are described in [EXPLAIN_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/EXPLAIN_demo.ipynb), and PREDICT queries are described in [PREDICT_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/PREDICT_demo.ipynb). Here, we describe PREDICT queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

**To experiment with an executable version of this notebook, [load it in Google Colaboratory](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/Multi%20intermediate%20nodes%20query.ipynb).**

## Step 0: Load BioThings Explorer modules

Install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).

In [ ]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [8]:
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection

## Step 1: Find representation of "Anisindione" in BTE

In this step, BioThings Explorer translates our query string "Anisindioine"  into BioThings objects, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [2]:
ht = Hint()
anisindione = ht.query("Anisindione")['ChemicalSubstance'][0]

anisindione

{'chembl': 'CHEMBL712',
 'drugbank': 'DB01125',
 'name': 'Anisindione',
 'pubchem': 2197,
 'umls': 'C0051919',
 'chebi': 'CHEBI:133809',
 'display': 'chembl(CHEMBL712) drugbank(DB01125) name(Anisindione) pubchem(2197) umls(C0051919) chebi(CHEBI:133809) ',
 'type': 'ChemicalSubstance',
 'primary': {'identifier': 'chembl',
  'cls': 'ChemicalSubstance',
  'value': 'CHEMBL712'}}

## Step 2: Find phenotypes that are associated with Anisindione through Gene and DiseaseOrPhenotypicFeature as intermediate nodes

In this section, we find all paths in the knowledge graph that connect Anisindione to any entity that is a phenotypic feature.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

In [3]:
fc = FindConnection(input_obj=anisindione, 
                    output_obj='PhenotypicFeature', 
                    intermediate_nodes=['Gene', 'DiseaseOrPhenotypicFeature'])

In [4]:
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'Anisindione' and 'PhenotypicFeature'. Paths will have 2 intermediate node.

Intermediate node #1 will have these type constraints: Gene

Intermediate node #2 will have these type constraints: DiseaseOrPhenotypicFeature



========== QUERY #1 -- fetch all Gene entities linked to Anisindione ==========

==== Step #1: Query path planning ====

Because Anisindione is of type 'ChemicalSubstance', BTE will query our meta-KG for APIs that can take 'ChemicalSubstance' as input and 'Gene' as output

BTE found 4 apis:

API 1. dgidb_chemical2gene(1 API call)
API 2. chembl_drug_mechanism(1 API call)
API 3. semmedgene(6 API calls)
API 4. mychem.info(2 API calls)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 4.2: http://mychem.info/v1/query (POST "q=DB01125&scopes=drugbank.id&fields=drugbank.enzymes,drugbank.targets&species=

In [7]:
fc.display_table_view()

,input,input_type,pred1,pred1_source,pred1_api,pred1_pubmed,node1_type,node1_name,node1_id,pred2,pred2_source,pred2_api,pred2_pubmed,node2_type,node2_name,node2_id,pred3,pred3_source,pred3_api,pred3_pubmed,output_type,output_name,output_id
0,name:ANISINDIONE,ChemicalSubstance,target,drugcentral,mychem.info,None,Gene,symbol:ALOX5,symbol:ALOX5,associatedWith,DISEASES,DISEASES,None,DiseaseOrPhenotypicFeature,mondo:DOID:162,mondo:DOID:162,associatedWith,biolink,biolink_disease2phenotype,None,PhenotypicFeature,hp:HP:0002664,hp:HP:0002664
1,name:ANISINDIONE,ChemicalSubstance,target,dgidb,dgidb_chemical2gene,None,Gene,symbol:GC,symbol:GC,associatedWith,DISEASES,DISEASES,None,DiseaseOrPhenotypicFeature,mondo:DOID:162,mondo:DOID:162,associatedWith,biolink,biolink_disease2phenotype,None,PhenotypicFeature,hp:HP:0002664,hp:HP:0002664
2,name:ANISINDIONE,ChemicalSubstance,target,drugcentral,mychem.info,None,Gene,symbol:ALOX5,symbol:ALOX5,associatedWith,DISEASES,DISEASES,None,DiseaseOrPhenotypicFeature,mondo:DOID:162,mondo:DOID:162,associatedWith,biolink,biolink_disease2phenotype,None,PhenotypicFeature,hp:HP:0002664,hp:HP:0002664
3,name:ANISINDIONE,ChemicalSubstance,target,dgidb,dgidb_chemical2gene,None,Gene,symbol:GC,symbol:GC,associatedWith,DISEASES,DISEASES,None,DiseaseOrPhenotypicFeature,mondo:DOID:162,mondo:DOID:162,associatedWith,biolink,biolink_disease2phenotype,None,PhenotypicFeature,hp:HP:0002664,hp:HP:0002664
4,name:ANISINDIONE,ChemicalSubstance,target,drugcentral,mychem.info,None,Gene,symbol:GGCX,symbol:GGCX,associatedWith,DISEASES,DISEASES,None,DiseaseOrPhenotypicFeature,name:THROMBOTIC DISEASE,name:THROMBOTIC DISEASE,associatedWith,biolink,biolink_disease2phenotype,None,PhenotypicFeature,hp:HP:0002664,hp:HP:0002664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19355,name:ANISINDIONE,ChemicalSubstance,target,dgidb,dgidb_chemical2gene,None,Gene,symbol:GC,symbol:GC,causes,semmeddisease,semmeddisease,None,DiseaseOrPhenotypicFeature,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,associatedWith,hpo,mydisease.info,None,PhenotypicFeature,hp:HP:0002446,hp:HP:0002446
19356,name:ANISINDIONE,ChemicalSubstance,target,dgidb,dgidb_chemical2gene,None,Gene,symbol:GC,symbol:GC,causes,semmeddisease,semmeddisease,None,DiseaseOrPhenotypicFeature,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,associatedWith,hpo,mydisease.info,None,PhenotypicFeature,hp:HP:0000478,hp:HP:0000478
19357,name:ANISINDIONE,ChemicalSubstance,target,dgidb,dgidb_chemical2gene,None,Gene,symbol:GC,symbol:GC,causes,semmeddisease,semmeddisease,None,DiseaseOrPhenotypicFeature,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,associatedWith,hpo,mydisease.info,None,PhenotypicFeature,hp:HP:0001276,hp:HP:0001276
19358,name:ANISINDIONE,ChemicalSubstance,target,dgidb,dgidb_chemical2gene,None,Gene,symbol:GC,symbol:GC,causes,semmeddisease,semmeddisease,None,DiseaseOrPhenotypicFeature,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,name:MITOCHONDRIAL DNA DEPLETION SYNDROME 4A,associatedWith,hpo,mydisease.info,None,PhenotypicFeature,hp:HP:0003678,hp:HP:0003678
